In [3]:
from sage.arith.misc import CRT

# Small-message attack

If $c = m^3 < N$, then it's easy to find the cube root of $c$.

In [33]:
keys = []

while len(keys) < 3:
    p = random_prime(2**150)
    q = random_prime(2**150)
    n = p*q
    totient = (p-1)*(q-1)
    e = 3
    if gcd(e, totient) != 1:
        continue
    d = inverse_mod(e, totient)

    keys.append({
        "p": p,
        "q": q,
        "n": n,
        "e": e,
        "d": d
    })


We convert the message to a big-endian integer. Note how all three encryptions are identical!

In [11]:
m = int.from_bytes("RPISEC".encode("utf-8"), "big")

ctexts = []

for key in keys:
    c = pow(m, key["e"], key["n"]) 
    ctexts.append(c)

for i, c in enumerate(ctexts):
    print("Ciphertext {0}: {1}".format(i, c))

Ciphertext 0: 741335105320677165077710424156151745895899
Ciphertext 1: 741335105320677165077710424156151745895899
Ciphertext 2: 741335105320677165077710424156151745895899


This makes decryption trivial

In [32]:
for i, c in enumerate(ctexts):
    # note -- c is an IntegerMod, not an Integer
    m = Integer(c).nth_root(3)
    # and now we just want a python integer
    data = int(m).to_bytes(6, "big").decode("utf-8")
    print("Message {0}: {1}".format(i, data))

Message 0: RPISEC
Message 1: RPISEC
Message 2: RPISEC


# Håstad Broadcast

Even if $m^3 < N$, we can still have problems!

In [63]:
m = int.from_bytes("A bit of a bigger message".encode("utf-8"), "big")

ctexts = []

for key in keys:
    c = pow(m, key["e"], key["n"]) 
    ctexts.append(c)

for i, c in enumerate(ctexts):
    print("Ciphertext {0}: {1}".format(i, c))

Ciphertext 0: 23371631345177228513179454558643083458124168553718700896780318741729030204472038343147173
Ciphertext 1: 79855603254367778985658289947212841881511440584010531350080344552739327784228071724647
Ciphertext 2: 151136270436655689935129687634376002718339697808694747675515448057165653694354967169757559


Note how the ciphertexts are distinct.

We apply the *Chinese Remainder Theorem (CRT)* to find a value of $C$ such that $C \equiv C_i \bmod N_i$

Thus, $C = M^3 \bmod (N_1 N_2 N_3)$, and $M^3 < N_1 N_2 N_3$

In [67]:
residues = [Integer(c) for c in ctexts]
moduli = [key["n"] for key in keys]
C = CRT(residues, moduli)
m = int(C.nth_root(3))
length = ceil(log(m,2).n()/8)
message = m.to_bytes(length, "big").decode("utf-8")

print(message)

A bit of a bigger message


Whilst valid, using $e=3$ can lead to a lot of weird (and insecure) situations!